In [36]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True
    )

prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 사람에게 도움을 주는 AI야, 물어본 언어로 대답해"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{question}"),
])

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)

invoke_chain("내 이름은 원국")

content='안녕하세요, 원국님! 무엇을 도와드릴까요?'


In [37]:
invoke_chain("난 강남에 살아")

content='강남은 서울의 한 지역으로 유명하죠! 강남에서는 무엇을 주로 하시나요? 혹시 궁금한 정보가 있거나 도움이 필요한 것이 있으면 말씀해주세요.'


In [38]:
invoke_chain(question="자곡동 알아 ? 강남같지 않아 시골 같아")

content='네, 자곡동은 강남구에 위치한 지역이죠. 강남과는 조금 다른 분위기를 가지고 있을 수 있습니다. 시골 같은 조용하고 평화로운 분위기가 좋을 수도 있겠죠. 자곡동에서의 생활이 어떤가요? 무엇이든 물어보세요.'


In [40]:
invoke_chain(question="내 이름 기억하고 있니? 어디살아?")

content='네, 원국님 맞죠? 강남에 사신다고 하셨죠? 어떻게 도와드릴까요?'
